In [ ]:
with open("names.txt", "r") as file:
    words = file.read().splitlines()

In [ ]:
words[:10]

In [ ]:
len(words)

In [ ]:
b = {}
for w in words:
    chs = ["<S>"] + list(w) + ["<E>"]

    for ch1,ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1


In [ ]:
sorted(b.items(), key=lambda item: -item[1])

In [ ]:
%pip install matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
%pip install torch torchvision torchaudio

In [ ]:
import torch

In [ ]:
N = torch.zeros((27, 27), dtype=torch.int32)

In [ ]:
chars = sorted(list(set(''.join(words))))
stoi = {s : i+1 for i, s in enumerate(chars)}
stoi["."] = 0
itos = {i: s for s, i in stoi.items()}

In [ ]:
N = torch.zeros((27, 27), dtype=torch.int32)
for w in words:
    chs = ["."] + list(w) + ["."]

    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1, ix2] += 1

In [ ]:
plt.figure(figsize=(16, 16))
plt.imshow(N, cmap="Blues")
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j]
        plt.text(j,i,chstr, ha="center", va="bottom", color="black", fontsize=8)
        plt.text(j,i,N[i,j].item(), ha="center", va="top", color="black", fontsize=8)
        
plt.axis("off")

In [ ]:
N[0]


In [ ]:
p = N[0].float()
p = p / p.sum()
p

In [ ]:
g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
itos[ix]


In [ ]:
P = N.float()
P = P / P.sum(dim=1, keepdim=True)

In [ ]:
g = torch.Generator().manual_seed(2147483647)
for i in range(20):
    out = []
    ix = 0
    while True:
        p = P[ix]
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
        
    print(''.join(out))
    